In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('article_leman.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
df

,category,text,lemmatized_text
0,sport,Szijjártó Péter az öltözőben gratulált a válog...,szijjártó péter öltöző gratulál válogatott ját...
1,sport,Szilágyi Áron bronzérmes lett a vívó-világbajn...,szilágyi ár bronzérmes hat tus vezet elődöntő ...
2,gazdasag,Ifjuságkutató Intézet: a fiatalok többsége Mag...,ifjuságkutató intézet fiatal többség magyarors...
3,sport,A négyszeres világbajnok angol klasszist is ki...,négyszeres világbajnok angol klasszis kiment r...
4,kulfold,Eltávolították a Twitter feliratot A rendőrség...,eltávolít twitter felirat rendőrség megjelenik...
...,...,...,...
18834,belfold,Gömbös Gyula egykori lakóhelyén működik a Part...,gömbös gyula egykori lakóhely működik partizán...
18835,kulfold,Ukrán gyerekek háborús naplóit mutatták be Ams...,ukrán gyerek háborús napló mutat amszterdam há...
18836,belfold,Publicus Intézet: a magyarok negyede azt sem t...,publicus intézet magyar negyed tud hol rendez ...
18837,belfold,"A kultúra mindig változik Száz éve Krúdy, most...",kultúra változik száz év krúdy mostanra krúbi


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# 1. TF-IDF kombináció
df['combined'] = df['text'] + ' ' + df['lemmatized_text']

vectorizer = TfidfVectorizer(max_features=20000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['combined'])

# 2. Célváltozó előkészítése
le = LabelEncoder()
y = le.fit_transform(df['category'])
y_cat = to_categorical(y)

# 3. Tanító és teszt halmaz
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, stratify=y, test_size=0.2, random_state=42)

# 4. Súlyozás a kiegyensúlyozatlan osztályokhoz
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
weight_dict = dict(enumerate(class_weights))

# 5. Neurális háló modell
model = Sequential([
    Dense(256, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_cat.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# 6. Tanítás
model.fit(X_train.toarray(), y_train, epochs=10, batch_size=64, validation_split=0.2, class_weight=weight_dict)

# 7. Értékelés
loss, acc = model.evaluate(X_test.toarray(), y_test)
print(f"Pontosság: {acc:.3f}")


Epoch 1/10
189/189 [==============================] - 9s 46ms/step - loss: 1.3764 - accuracy: 0.6270 - val_loss: 0.9178 - val_accuracy: 0.7068
Epoch 2/10
189/189 [==============================] - 8s 44ms/step - loss: 0.4060 - accuracy: 0.8768 - val_loss: 0.8658 - val_accuracy: 0.7221
Epoch 3/10
189/189 [==============================] - 9s 46ms/step - loss: 0.1096 - accuracy: 0.9650 - val_loss: 0.9320 - val_accuracy: 0.7317
Epoch 4/10
189/189 [==============================] - 9s 47ms/step - loss: 0.0386 - accuracy: 0.9891 - val_loss: 1.0341 - val_accuracy: 0.7280
Epoch 5/10
189/189 [==============================] - 9s 50ms/step - loss: 0.0203 - accuracy: 0.9948 - val_loss: 1.1098 - val_accuracy: 0.7297
Epoch 6/10
189/189 [==============================] - 9s 48ms/step - loss: 0.0125 - accuracy: 0.9968 - val_loss: 1.1773 - val_accuracy: 0.7264
Epoch 7/10
189/189 [==============================] - 7s 35ms/step - loss: 0.0087 - accuracy: 0.9974 - val_loss: 1.2265 - val_accuracy: 0.7257

In [10]:
model.save("model_nev.h5")


C:\Users\KomPhone\projects\ml\cubix_ml_2\.venv\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

model = load_model("model_nev.h5")

text = "50 milliárdért szerzi meg a Mol a BME fenntartóját"
text_lemmatized = ""
combined = text + " " + text_lemmatized

X_new = vectorizer.transform([combined])
X_new_dense = X_new.toarray()  # ← ez oldja meg a hibát

y_pred = model.predict(X_new_dense)
pred_label = np.argmax(y_pred, axis=1)

print("Predikált osztály index:", pred_label[0])
print("Kategória:", le.inverse_transform(pred_label)[0])


1/1 [==============================] - 0s 26ms/step
Predikált osztály index: 2
Kategória: gazdasag
